<a href="https://colab.research.google.com/github/lechemrc/DS-Unit-2-Applied-Modeling/blob/master/module2/assignment_applied_modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 2

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Plot the distribution of your target. 
    - Classification problem: Are your classes imbalanced? Then, don't use just accuracy.
    - Regression problem: Is your target skewed? If so, let's discuss in Slack.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline?
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - _**[A Kaggle Master Explains Gradient Boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/)**_
  - [_An Introduction to Statistical Learning_](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf) Chapter 8
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)
  - _**[Boosting](https://www.youtube.com/watch?v=GM3CDQfQ4sw) (2.5 minute video)**_

### Colab Setup

In [71]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module1')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling
 * branch            master     -> FETCH_HEAD
Already up to date.


### Important Imports

In [0]:
# libraries and math functions
import pandas as pd
import numpy as np
import pandas_profiling
from scipy.io import arff # for loading .arff file
from scipy.stats import randint, uniform

# imports for pipeline and regression
import category_encoders as ce
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import validation_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from ipywidgets import interact, fixed

# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### Importing dataset

Dataset has largely been cleaned previously 

In [73]:
df = pd.read_csv('https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Autism%20Screening%20for%20Children/csv_result-Autism-Child-Data.csv', na_values='?')
print(df.shape)
df.head()

(292, 22)


,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,0,0,1,1,0,1,0,0,6.0,m,Others,no,no,Jordan,no,5,4-11 years,Parent,NO
1,2,1,1,0,0,1,1,0,1,0,0,6.0,m,Middle Eastern,no,no,Jordan,no,5,4-11 years,Parent,NO
2,3,1,1,0,0,0,1,1,1,0,0,6.0,m,NaN,no,no,Jordan,yes,5,4-11 years,NaN,NO
3,4,0,1,0,0,1,1,0,0,0,1,5.0,f,NaN,yes,no,Jordan,no,4,4-11 years,NaN,NO
4,5,1,1,1,1,1,1,1,1,1,1,5.0,m,Others,yes,no,United States,no,10,4-11 years,Parent,YES


### Data Wrangling

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Autism%20Screening%20for%20Children/csv_result-Autism-Child-Data.csv', na_values='?')

def data_wrangle(df):
  ''' cleaning the data with one function'''

  # null values
  df = df.fillna(value='unspecified')

  # Dropping columns with single value or counfounding values
  df = df.drop(['age_desc', 'result'], axis=1)

  # Dropping id column to prevent obfuscation of data
  df = df.drop('id', axis=1)

  # Cleaning column names
  df = df.rename(columns={'jundice':'born_jaundice', 
                          'austim':'family_pdd', 
                          'contry_of_res':'country', 
                          'used_app_before':'prior_screening'})
  
  # Changing the country column values to 'other' if there are less
  # than 5 instances in the df
  frequencies = df['country'].value_counts()
  condition = frequencies <= 5
  mask = frequencies[condition].index
  mask_dict = dict.fromkeys(mask, 'other')

  df['country'] = df['country'].replace(mask_dict)

  # renaming values for clarity
  df['relation'] = df['relation'].replace(
      {'self':'Self', 
       'Health care professional':'Healthcare Professional', 
       'unspecified':'Unspecified'})
  
  df['ethnicity'] = df['ethnicity'].replace({'Pasifika':'Pacifica', 
                                           'Others':'unspecified'})

  return df

In [0]:
df = data_wrangle(df)

### Regression and Analysis

In [108]:
# Setting target and features
target = 'Class/ASD'

# Dropping 'result' and 'age' as they seem to be confounding and not helpful
features = df.columns.drop(target)

X = df[features]
y = df[target]

# Train / Test split
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42)

# Train / Val split 
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, 
    stratify=y_trainval, random_state=42)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape 

((174, 18), (174,), (59, 18), (59,), (59, 18), (59,))

In [109]:
features

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'ethnicity', 'born_jaundice', 'family_pdd', 'country',
       'prior_screening', 'relation'],
      dtype='object')

#### Majority Class Accuracy

In [0]:
pd.options.display.float_format = None

In [111]:
y_train.value_counts(normalize=True)

NO     0.517241
YES    0.482759
Name: Class/ASD, dtype: float64

In [112]:
# Accuracy score using the majority class
majority_class = y_train.mode()[0]
y_pred = np.full_like(y_val, fill_value=majority_class)
accuracy_score(y_val, y_pred)

0.5084745762711864

#### Random Forest Classifier with One Hot Encoding

In [118]:
random_forest = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
)

random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
print('Validation Accuracy:', random_forest.score(X_val, y_val), 
      '\nTest Accuracy:', random_forest.score(X_test, y_test))

Validation Accuracy: 0.8813559322033898 
Test Accuracy: 0.9322033898305084


### Feature Importances

In [114]:
X_train.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'ethnicity', 'born_jaundice', 'family_pdd', 'country',
       'prior_screening', 'relation'],
      dtype='object')

In [116]:
rf.feature_importances_

array([0.03722184, 0.03063948, 0.05597889, 0.14128943, 0.08375588,
       0.0583743 , 0.02197393, 0.10774933, 0.05423606, 0.08915127,
       0.01007553, 0.01035952, 0.008754  , 0.01345822, 0.00359328,
       0.002314  , 0.0053307 , 0.00987898, 0.00482236, 0.01378238,
       0.01208756, 0.01206801, 0.01034159, 0.00922778, 0.00966956,
       0.00159414, 0.00085258, 0.00046015, 0.00433159, 0.00187545,
       0.00591074, 0.01140805, 0.01177434, 0.0077852 , 0.00824433,
       0.00777664, 0.00745657, 0.01604227, 0.00276233, 0.00647191,
       0.00533646, 0.00361401, 0.00286322, 0.01240293, 0.00862961,
       0.00475521, 0.00437327, 0.00454252, 0.00270443, 0.01590499,
       0.0109335 , 0.00756735, 0.00549233])

In [119]:
# Get feature importances
rf = random_forest.named_steps['randomforestclassifier']
importances = pd.Series(rf.feature_importances_, X_train.columns)

# Plot feature importances
%matplotlib inline
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh(color='grey');

ValueError: ignored